In [36]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

class SinDataset(Dataset):
    def __init__(self, x, included_timesteps, shift):
        """
        Initialize the dataset with the data.

        Args:
            data (list or array-like): List of numbers to use as data.
        """
        self.data = x
        self.included_timesteps = included_timesteps
        self.shift = shift

    def __len__(self):
        """
        Return the number of samples in the dataset.
        """
        return len(self.data)-self.included_timesteps

    def __getitem__(self, idx):
        """
        Fetch the data and its label at index `idx`.

        Args:
            idx (int): Index of the sample to retrieve.
        
        Returns:
            tuple: (data, label) where label is the square of the data.
        """
        x = np.sin(self.data[idx:idx+self.included_timesteps] + self.shift)  # Get the input data
        y = np.sin(self.data[idx+self.included_timesteps] + self.shift)         # Label is the square of the input
        return x, y


## Loading data

In [70]:

included_timesteps = 50
timeline = np.linspace(0, 60, 500)
x1 = SinDataset(timeline, included_timesteps, 0)
x2 = SinDataset(timeline, included_timesteps, 1)
dataloader1 = DataLoader(x1, batch_size=1)
dataloader2 = DataLoader(x2, batch_size=1)

In [73]:
for (x1, y1), (x2, y2) in zip(dataloader1, dataloader2):
    print(x1.shape, y1.shape, x1.shape, y1.shape)
    x = torch.stack((x1, x2), dim=2).float()
    y = torch.stack((y1, y2), dim=1).float()
    print(x.shape, y.shape)
    print(x.dtype, y)
    break

torch.Size([1, 50]) torch.Size([1]) torch.Size([1, 50]) torch.Size([1])
torch.Size([1, 50, 2]) torch.Size([1, 2])
torch.float32 tensor([[-0.2679,  0.6660]])


## Setting up Network

In [74]:
class Net(nn.Module):
    def __init__(self, n_neurons, input_shape):
        super(Net, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=input_shape, hidden_size=n_neurons, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=input_shape, hidden_size=n_neurons, batch_first=True)

        self.fc1 = nn.Linear(n_neurons, n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.fc = nn.Linear(2*n_neurons, 2)
    
    def forward(self, x):
        _, (h_n1, _) = self.lstm1(x[:, :, 0])
        _, (h_n2, _) = self.lstm2(x[:, :, 1])
        out = torch.cat((h_n1, h_n2))
        out = torch.flatten(out)
        out = self.fc(out)
        return out

# Training loop

In [75]:
n_neurons = 12
epochs = 4

model = Net(n_neurons, included_timesteps)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_curve = []

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

for epoch in range(epochs):
    loss_total = 0
    model.train()

    for (x1, y1), (x2, y2) in zip(dataloader1, dataloader2):
        optimizer.zero_grad()
        x = torch.stack((x1, x2), dim=2).float().to(device)
        y = torch.flatten(torch.stack((y1, y2), dim=1).float()).to(device)

        predictions = model(x)
        loss = loss_function(predictions, y)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch}: {loss_total=}')


Epoch 0: loss_total=45.018419770382025
Epoch 1: loss_total=1.571711075046096
Epoch 2: loss_total=6.952122555448568
Epoch 3: loss_total=0.6216401294154821


In [76]:
# Predicting stuff
for (x1, y1), (x2, y2) in zip(dataloader1, dataloader2):
    x = torch.stack((x1, x2), dim=2).float()
    y = torch.flatten(torch.stack((y1, y2), dim=1).float())
    break

In [72]:
plt.plot(x[0])

In [77]:
prediction = []
for n in range(100):
  pred = model(x)
  y = torch.clone(x[:, 1:, :])
  new_x = pred.reshape((1, 1, 2))
  x = torch.cat((y, new_x), axis=1)
  prediction.append(new_x[0, 0, :].detach().numpy())

In [79]:
plt.plot(np.array(prediction).T[1])
plt.plot(np.array(prediction).T[0])

In [21]:
y.shape

torch.Size([1, 19, 2])

In [30]:
x[0, :, 0]

tensor([1.1688, 1.1697, 1.1706, 1.1717, 1.1729, 1.1738, 1.1744, 1.1748, 1.1748,
        1.1744, 1.1738, 1.1731, 1.1721, 1.1711, 1.1703, 1.1695, 1.1689, 1.1685,
        1.1683, 1.1681], grad_fn=<SelectBackward0>)

In [24]:
x.shape

torch.Size([1, 20, 2])